In [1]:
from ortools.sat.python import cp_model
import time
import pandas
import numpy as np
import itertools
import pickle

Parameters

In [2]:
params = {
    'num_personnel': 18, 
    'max_trips': 3, 
    'num_points_start': 2, 
    'num_points_holding': 3, 
    'num_points_destination': 2, 
    'num_vehicles_alpha': 2, 
    'num_vehicles_beta': 2, 
    'vehicle_capacity': 3, 
    'loading_time': 0, 
}

Initialise model

In [3]:
from planck_optimiser import Model

model = Model(params)

Route generation

In [4]:
model.generate_route()
model.routes

,S1,S2,H1,H2,H3,D1,D2
S1,999,999,8,9,6,999,999
S2,999,999,6,10,7,999,999
H1,8,6,999,999,999,5,8
H2,9,10,999,999,999,5,5
H3,6,7,999,999,999,10,10
D1,999,999,5,5,10,999,999
D2,999,999,8,5,10,999,999


Variables

In [5]:
model.generate_personnel_route()
model.generate_vehicle_route()
model.generate_node_visitation_vehicle()
model.generate_node_visitation_personnel()
model.generate_arrival_vehicle()
model.generate_arrival_personnel()
model.generate_waiting()

Constraints

In [6]:
## i. Vehicle arrives and departs from same HOLDING node if node is visited during that trip
model.generate_vehicle_node_constraint()

## ii. Vehicle v departs from node i (START/DESTINATION) in trip r+1, if trip exsits, if it arrives at node i in trip r
model.generate_vehicle_next_trip_constraint()

## iii. Only 1 start<->holding route / holding<->destination route per trip per vehicle
model.generate_vehicle_trip_limit_constraint()

## iv. Personnel arrives and departs from same HOLDING node if node is visited
model.generate_personnel_node_constraint()

## v. Personnel visits node only if it boards the vehicle
model.generate_board_vehicle_constraint()

## vi. Personnel departs from start node if it is stationed there
model.generate_personnel_start_constraint()

## vii. Only 1 start->holding & holding->destination per personnel
model.generate_personnel_trip_limit_constraint()

## viii. Vehicle capacity constraints
model.generate_vehicle_capacity_constraint()

## ix. Arrival time at START/DESTINATION = Arrival time at HOLDING + Loading time + Waiting time + 
##     Travel time if vehicle v takes route during trip r
model.generate_vehicle_arrival_constraint()

## x. Arrival time of trip r+1 of vehicle v at HOLDING = Arrival time at START/DESTINATION of trip r + 
##    Loading time + Travel time if route taken
model.generate_holding_arrival_constraint()

## xi. Arrival time of trip 0 of vehicle v at HOLDING = Loading time + Waiting time + 
##     Travel time if route taken
model.generate_holding_arrival_0_constraint()

## xii. Arrival time of personnel n at node i = Arrival time of trip r of vehicle v at node i if 
##      personnel boards it
model.generate_personnel_arrival_constraint()

## xiii. Personnel can only board trip r of alpha vehicle if its arrival time at node i larger than 
##       arrival time of trip r of beta vehicle at node i
model.generate_personnel_holding_constraint()


### Other constraints ###

## Vehicle v arrives at node i for the same number of trips that depart from node i (start/destination)
# model.generate_vehicle_trip_cycle_constraint()

## Subsequent trips of vehicle r can only take place if prior trip of the same route is taken
# model.generate_vehicle_subsequent_trip_constraint()

Initial state

In [7]:
## All personnel must visit respective START, HOLDING and DESTINATION
for n in range(model.num_personnel):
    respective = n % 3
    model.Add(model.node_p[(n, model.points_holding[respective])] == 1)
    if n < model.num_personnel // 2:
        respective = 0
    else:
        respective = 1
    model.Add(model.node_p[(n, model.points_start[respective])] == 1)
    model.Add(model.node_p[(n, model.points_destination[respective])] == 1)

Objectives

In [8]:
total_arrival_time = sum([model.arrival_p[(n, i)] for i in model.points_destination for n in range(model.num_personnel)])

Optimisation

In [9]:
model.Minimize(total_arrival_time)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('optimal objective value:', solver.ObjectiveValue())
elif status == cp_model.FEASIBLE:
    print('feasible objective value:', solver.ObjectiveValue())
elif status == cp_model.INFEASIBLE:
    print("no solution found")
else:
    print("error, check the status and log of the solve")
print(solver.ResponseStats().split('\n')[12])

optimal objective value: 531.0
walltime: 1.87471


Results

In [10]:
print("\nnode_v")
for k, v in model.node_v.items():
    print(k, solver.Value(v))

print("\nnode_p")
test = pandas.DataFrame(index=range(model.num_personnel), columns=model.points)
for i in model.node_p:
    test.loc[i[0], i[1]] = solver.Value(model.node_p[i])
print(test)

print("\nvar_q")
for k, v in model.var_q.items():
    print(k, solver.Value(v))
print("\nvar_p")
for k, v in model.var_p.items():
    print(k, solver.Value(v))

print("\nvar_y")
for k, v in model.var_y.items():
    print(k, solver.Value(v))
print("\nvar_x")
for k, v in model.var_x.items():
    print(k, solver.Value(v))

print("\narrival_v")
for k, v in model.arrival_v.items():
    print(k, solver.Value(v))

print("\narrival_p")
for k, v in model.arrival_p.items():
    print(k, solver.Value(v))


node_v
('Va1', 0, 'H1') 1
('Va1', 0, 'H2') 0
('Va1', 0, 'H3') 0
('Va1', 0, 'D1') 0
('Va1', 0, 'D2') 0
('Va1', 1, 'H1') 0
('Va1', 1, 'H2') 0
('Va1', 1, 'H3') 1
('Va1', 1, 'D1') 1
('Va1', 1, 'D2') 0
('Va1', 2, 'H1') 0
('Va1', 2, 'H2') 1
('Va1', 2, 'H3') 0
('Va1', 2, 'D1') 0
('Va1', 2, 'D2') 1
('Va2', 0, 'H1') 1
('Va2', 0, 'H2') 0
('Va2', 0, 'H3') 0
('Va2', 0, 'D1') 0
('Va2', 0, 'D2') 0
('Va2', 1, 'H1') 0
('Va2', 1, 'H2') 1
('Va2', 1, 'H3') 0
('Va2', 1, 'D1') 0
('Va2', 1, 'D2') 1
('Va2', 2, 'H1') 0
('Va2', 2, 'H2') 0
('Va2', 2, 'H3') 1
('Va2', 2, 'D1') 0
('Va2', 2, 'D2') 1
('Vb1', 0, 'S1') 0
('Vb1', 0, 'S2') 0
('Vb1', 0, 'H1') 1
('Vb1', 0, 'H2') 0
('Vb1', 0, 'H3') 0
('Vb1', 1, 'S1') 0
('Vb1', 1, 'S2') 1
('Vb1', 1, 'H1') 0
('Vb1', 1, 'H2') 0
('Vb1', 1, 'H3') 1
('Vb1', 2, 'S1') 1
('Vb1', 2, 'S2') 0
('Vb1', 2, 'H1') 0
('Vb1', 2, 'H2') 1
('Vb1', 2, 'H3') 0
('Vb2', 0, 'S1') 0
('Vb2', 0, 'S2') 0
('Vb2', 0, 'H1') 1
('Vb2', 0, 'H2') 0
('Vb2', 0, 'H3') 0
('Vb2', 1, 'S1') 0
('Vb2', 1, 'S2') 1
('Vb

Visualise

In [11]:
schedule_p = pandas.DataFrame(index=range(model.num_personnel), columns=['P_S', 'P_H', 'P_D', 'V_A', 'V_B', 't_H', 't_D'])

## Node visited by personnel
for k, v in model.node_p.items():
    if solver.Value(v) == 1:
        node_type = 'P_' + k[1][0]
        schedule_p.loc[k[0], node_type] = k[1]

## Vehicle taken by personnel
for k, v in model.var_y.items():
    if solver.Value(v) == 1:
        vehicle_type = 'V_' + k[1][1].upper()
        schedule_p.loc[k[0], vehicle_type] = k[1]
for k, v in model.var_x.items():
    if solver.Value(v) == 1:
        vehicle_type = 'V_' + k[1][1].upper()
        schedule_p.loc[k[0], vehicle_type] = k[1]

## Arrival time of personnel
for k, v in model.arrival_p.items():
    if solver.Value(v):
        arrival_type = 't_' + k[1][0]
        schedule_p.loc[k[0], arrival_type] = solver.Value(v)

departure_p = {}
waiting_p = {}
for n in range(params['num_personnel']):
    departure_p[n] = schedule_p.loc[n, 't_D'] - model.routes.loc[schedule_p.loc[n, 'P_H'], schedule_p.loc[n, 'P_D']]
    waiting_p[n] = departure_p[n] - schedule_p.loc[n, 't_H']
schedule_p['wait'] = waiting_p.values()

schedule_p

,P_S,P_H,P_D,V_A,V_B,t_H,t_D,wait
0,S1,H1,D1,Va1,Vb1,8,13,0
1,S1,H2,D1,Va1,Vb1,38,43,0
2,S1,H3,D1,Va2,Vb2,37,47,0
3,S1,H1,D1,Va1,Vb1,8,13,0
4,S1,H2,D1,Va1,Vb1,38,43,0
5,S1,H3,D1,Va2,Vb2,37,47,0
6,S1,H1,D1,Va1,Vb1,8,13,0
7,S1,H2,D1,Va1,Vb1,38,43,0
8,S1,H3,D1,Va2,Vb2,37,47,0
9,S2,H1,D2,Va2,Vb2,6,14,0


In [12]:
rows = model.vehicles_beta + model.vehicles_alpha
cols = list(itertools.chain(*[[f'trip{i}_a', f'T{i}_a', f'load{i}_a', 
                               f'trip{i}_b', f'T{i}_b', f'load{i}_b'] for i in range(model.max_trips)]))
schedule_v = pandas.DataFrame('-', index=rows, columns=cols)

## Routes travelled by vehicle
for k, v in model.var_q.items():
    if solver.Value(v) == 1:
        if k[2][0] == 'S':
            trip_type = f'trip{k[1]}_' + 'a'
        else:
            trip_type = f'trip{k[1]}_' + 'b'
        schedule_v.loc[k[0], trip_type] = f'{k[2]} > {k[3]}'
for k, v in model.var_p.items():
    if solver.Value(v) == 1:
        if k[2][0] == 'D':
            trip_type = f'trip{k[1]}_' + 'a'
        else:
            trip_type = f'trip{k[1]}_' + 'b'
        schedule_v.loc[k[0], trip_type] = f'{k[2]} > {k[3]}'

## Arrival time of vehicle
for k, v in model.arrival_v.items():
    if solver.Value(v):
        if k[2][0] in ('S', 'D'):
            arrival_type = f'T{k[1]}_b'
        elif k[2][0] == 'H':
            arrival_type = f'T{k[1]}_a'
        schedule_v.loc[k[0], arrival_type] = solver.Value(v)

## Vehicle load
for k, v in model.var_q.items():
    if (solver.Value(v) == 1) and (k[2][0] == 'S'):
        total_load = ''
        for n in range(model.num_personnel):
            if solver.Value(model.var_y[(n,) + k]) == 1:
                total_load += f'n{n}, '
        if total_load:
            trip_type = f'load{k[1]}_a'
            schedule_v.loc[k[0], trip_type] = total_load[:-2]
for k, v in model.var_p.items():
    if (solver.Value(v) == 1) and (k[2][0] == 'H'):
        total_load = ''
        for n in range(model.num_personnel):
            if solver.Value(model.var_x[(n,) + k]) == 1:
                total_load += f'n{n}, '
        if total_load:
            trip_type = f'load{k[1]}_b'
            schedule_v.loc[k[0], trip_type] = total_load[:-2]

schedule_v

,trip0_a,T0_a,load0_a,trip0_b,T0_b,load0_b,trip1_a,T1_a,load1_a,trip1_b,T1_b,load1_b,trip2_a,T2_a,load2_a,trip2_b,T2_b,load2_b
Vb1,S1 > H1,8,"n0, n3, n6",H1 > S2,14,-,S2 > H3,21,"n11, n14, n17",H3 > S1,27,-,S1 > H2,38,"n1, n4, n7",H2 > S2,48,-
Vb2,S2 > H1,6,"n9, n12, n15",H1 > S2,12,-,S2 > H2,22,"n10, n13, n16",H2 > S1,32,-,S1 > H3,37,"n2, n5, n8",H3 > S1,43,-
Va1,D2 > H1,8,-,H1 > D1,13,"n0, n3, n6",D1 > H3,23,-,H3 > D2,33,"n11, n14, n17",D2 > H2,38,-,H2 > D1,43,"n1, n4, n7"
Va2,D1 > H1,5,-,H1 > D2,14,"n9, n12, n15",D2 > H2,19,-,H2 > D2,27,"n10, n13, n16",D2 > H3,37,-,H3 > D1,47,"n2, n5, n8"


In [13]:
## model_1: n=20, r=4, Q=3 >> obj=648, time=16min
# with open('model_1.pkl', 'wb') as f:
#     pickle.dump(model, f)

## model_2: n=14, r=4, Q=3 >> obj=471, time=9.3min
# with open('model_2.pkl', 'wb') as f:
#     pickle.dump(model, f)

## model_3: n=18, r=4, Q=3 >> obj=531, time=33min
# with open('model_3.pkl', 'wb') as f:
#     pickle.dump(model, f)

## model_4: n=18, r=3, Q=3 >> obj=531, time=3sec
# with open('model_4.pkl', 'wb') as f:
#     pickle.dump(model, f)



In [14]:
# with open('model_1.pkl', 'rb') as f:
#     model = pickle.load(f) 